<a href="https://colab.research.google.com/github/Alecia113/NLP-Lab/blob/main/lab4_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Recurrent Neural Networks + Sequence Modelling

In [18]:
#recurrent （复发的 蕊·科·run3·忒） sequence （计数，序数） （C 困·嘶） neural（牛肉）
#RNN ； seq2seq
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# you can GPU(cuda)/ just CPU

#RNN

In [19]:
'''
RNN 是一类神经网络。其中节点之间的连接形成沿着时间序列的有向图。 
时间手段跨时间分布。
将它与分布在空间中的网络进行比较。这可让他表现出时间动态行为。
RNN 源自前馈神经网络。
可以使用其内部状态（内存）来处理可变长度的输入序列

'''

'\nRNN 是一类神经网络。其中节点之间的连接形成沿着时间序列的有向图。 \n时间手段跨时间分布。\n将它与分布在空间中的网络进行比较。这可让他表现出时间动态行为。\nRNN 源自前馈神经网络。\n可以使用其内部状态（内存）来处理可变长度的输入序列\n\n'

#**predict the last character of the word** 预测单词的最后一个字符

**eg："wor"-->"d", "dee"-->"p"**

##有什么用呢？？？？？？

In [20]:
#这个模块的用意呢：输入单词。输入除了最后一个看，看能不能准确的匹配上最后一位

'''
1导包
2导入文本库 tip：有，直接换行就好了  [字母表]
encoding 编码 decoding解码  num_dic 让word 和index 对应起来
看下长度26
4这才开始导入文本。你想做测试的文本

5写函数了，开始要做拆解了(批量处理)
目的

# Make a batch to have sequence data for input and ouput
# wor -> X, d -> Y
# dee -> X, p -> Y

6开始循环，把除了最后一个【：-1】的字母的index都取出来
            # input data is:
        # wor       woo       dee     div      ...
        # [22, 14, 17] [22, 14, 14] [3, 4, 4] [3, 8, 21] ...


    再把最后一个单词取出来，把index 放到target里

    #制作one-hot np.eye 批量制作。然后第一个是0.所以index+1的位置是1.代表了这个元素
【将输入转为了一键编码】就不像之前我还得一个个编进去用*vec【0】
 # convert input to one-hot encoding.
        # if input is [3, 4, 4]:
        # [[ 0,  0,  0,  1,  0,  0,  0, ... 0]
        #  [ 0,  0,  0,  0,  1,  0,  0, ... 0]
        #  [ 0,  0,  0,  0,  1,  0,  0, ... 0]]
'''

'''
数据准备完开始设置hyerparameters
'''

'''
然后准备dropout 是一个全连接层，开始丢弃神经元了
'''

'\n然后准备dropout 是一个全连接层，开始丢弃神经元了\n'

In [21]:
#这个过程的数据准备还是很像CBOW的

import numpy as np
char_arr = ['a','b','c','d','e',
            'f','g','h','i','j',
            'k','l','m','n','o',
            'p','q','r','s','t',
            'u','v','w','x','y','z']

num_dic = {n:i for i,n in enumerate(char_arr)}  # {'a': 0, 'b': 1, 'c': 2, ..., 'j': 9, 'k', 10, ...}
dic_len = len(num_dic)

seq_data = ['word','wood','deep','dive','cold',
            'cool','load','love','kiss','kind']

def make_batch(seq_data):
  input_batch = []
  target_batch = []

  for seq in seq_data:
    input_data = [num_dic[n] for n in seq[:-1]] ##不取最后一个，把前面的字母对应的index取出来
    target = num_dic[seq[-1]]

    input_batch.append(np.eye(dic_len)[input_data])

    target_batch.append([target])  #就是输入的index ；target要加【】

  return input_batch, target_batch

#这个过程的数据准备还是很像CBOW的


In [22]:
# setting hyperparameters
learning_rate = 0.01
n_hidden = 64
total_epoch = 50

n_step = 3    #暂时的RNN序列长度 temporal length of sequences for RNN

n_input = dic_len  #26 输入向量的维度（dimension of input vector)
n_class = dic_len #类的数量 26 number of class



**dropout 让每个隐藏单元更强大； 并推动其自行创建有用的功能。不用依赖其他隐藏单元来纠正错误**

In [23]:
#感觉像是从一个全连接开始丢神经元。（还是全连接）
'''
1调包
建类别 调用nn模型开始建设RNN
    讲解RNN中参数： 
          batch_first 默认（default）False ； 代表输入和输出张量的提供方式为（seq_len, batch_size, feature)
          但是：我们需要True ：bec：我们要使用形状输入（batch_size, seq_len, feature)
            设置 num_layer = 2 我们将两个RNN 层堆叠在一起形成一个堆叠的RNN。
                              第一个RNN接收第一个RNN的输出，并计算最终结果。
          应用dropout（=0.2） aim：防止过拟合； 可更改 ； 【注意】可以用在RNN的每一层除了最后一层
          https://pytorch.org/docs/stable/generated/torch.nn.RNN.html
          官方解读

  forward function（函数）
   nn.RNN 有两个output
  1：形状张量（batch_size, seq_len, hidden_size)有点像True；包含每个时间步长t 的RNN的最后一层输出特征。
  2： 形状张量（num_layers * num_directions, batch, hidden_size) :包含 t = seq_len 的隐藏状态的张量
   我们只关心第一个输出
    nn.RNN: https://pytorch.org/docs/stable/generated/torch.nn.RNN.html#torch.nn.RNN


    我们仅仅要RNN输出功能中提取最后一个隐藏状态（from output features get(extract) last hidden state)
    最后的隐藏状态将信息从RNN信元（cell）随时间流逝中移走
    因此    基于最后一个隐藏状态的预测，不仅考虑当前时间'  步长的数据，还要考虑历史数据。（就是输出中包含很多类别的数据）
    来源复杂

输入然后得到的输出项（forward）

  3:设置模型，到GPU
  4: loss + optimizer

二

5模型建设完，开始准备输入数据
#https://blog.csdn.net/u014687517/article/details/90770132?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522161698050316780265445973%2522%252C%2522scm%2522%253A%252220140713.130102334.pc%255Fall.%2522%257D&request_id=161698050316780265445973&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~first_rank_v2~rank_v29-2-90770132.first_rank_v2_pc_rank_v29&utm_term=.view的用途
view 用法。
view 很像是reshape
a = torch.range(1, 25) #a是长度25的张量
#改成了5*5：a = a.view(5, 5)
或者：a = a.view(-1, 5)

（感觉就是将数据类型转变成为了张量。再传参一波，没做什么其他的感觉）

三
开始训练了epoch
然后经典三部 forward + backward + optimize
  #神经网络中经常会有
  作用:output 是通过神经网络最后一层softmax 函数作用之后的输出。
  
  例如：假设我们在分类问题中共有4类。 分别是0123； 假如output.data 第一行【1，000】（属于第一个类别的概率是1
  不加_, :输出则是1， （其中1是tensor） 加了之后输出变成0

  下划线+'，'的作用是：使预测返回的output.data行中最大数值所在位置代表的类别。
  _,predicted=torch.max(output.data,dim=1)

  【另】
    torch.max()返回两个值，一个是具体的value ；用下划线表示。第二个值是 value对应的index（predicted）
    下划线_ 表示的就是具体的value，也就是输出的最大值（可用其他名词代替下划线）
    数字1其实可以写为dim=1，这里简写为1
    dim = 1 输出行的最大值。 dim = 0 输出列的最大值
    #https://blog.csdn.net/weixin_48249563/article/details/111387501?utm_medium=distribute.pc_relevant.none-task-blog-BlogCommendFromMachineLearnPai2-1.control&dist_request_id=1328740.37292.16169815268198889&depth_1-utm_source=distribute.pc_relevant.none-task-blog-BlogCommendFromMachineLearnPai2-1.control


结论是损失和 训练准确度

四：
预测
设置flag 去计算模型； 会 turn off ：dropout 
将标志设置为评估模式，这将dropout退出。

训练完train_datasets之后，model要来测试样本了。在model(test_datasets)之前，需要加上model.eval(). 否则的话，有输入数据，即使不训练，它也会改变权值。这是model中含有batch normalization层所带来的的性质。
在做one classification的时候，训练集和测试集的样本分布是不一样的，尤其需要注意这一点。

[训练完数据要训练样本；为了让训练集和测试集的样本分开]（感觉更像是实现全连接的需要的感觉）
#https://blog.csdn.net/qq_38410428/article/details/101102075?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522161698276116780271557000%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=161698276116780271557000&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~top_click~default-2-101102075.first_rank_v2_pc_rank_v29&utm_term=model.eval

先把预测值取出来。
然后循环起来

pytorch中GPU与CPU的相互转化

深度学习中我们默认使用的是CPU，如果我们要使用GPU，需要使用.cuda将计算或者数据从CPU移动至GPU，

如果当我们需要在CPU上进行运算时，比如使用plt可视化绘图, 我们可以使用.cpu将计算或者数据转移至CPU.

然后开始拼接单词。
看拼出来的单词对不对

'''

"\n1调包\n建类别 调用nn模型开始建设RNN\n    讲解RNN中参数： \n          batch_first 默认（default）False ； 代表输入和输出张量的提供方式为（seq_len, batch_size, feature)\n          但是：我们需要True ：bec：我们要使用形状输入（batch_size, seq_len, feature)\n            设置 num_layer = 2 我们将两个RNN 层堆叠在一起形成一个堆叠的RNN。\n                              第一个RNN接收第一个RNN的输出，并计算最终结果。\n          应用dropout（=0.2） aim：防止过拟合； 可更改 ； 【注意】可以用在RNN的每一层除了最后一层\n          https://pytorch.org/docs/stable/generated/torch.nn.RNN.html\n          官方解读\n\n  forward function（函数）\n   nn.RNN 有两个output\n  1：形状张量（batch_size, seq_len, hidden_size)有点像True；包含每个时间步长t 的RNN的最后一层输出特征。\n  2： 形状张量（num_layers * num_directions, batch, hidden_size) :包含 t = seq_len 的隐藏状态的张量\n   我们只关心第一个输出\n    nn.RNN: https://pytorch.org/docs/stable/generated/torch.nn.RNN.html#torch.nn.RNN\n\n\n    我们仅仅要RNN输出功能中提取最后一个隐藏状态（from output features get(extract) last hidden state)\n    最后的隐藏状态将信息从RNN信元（cell）随时间流逝中移走\n    因此    基于最后一个隐藏状态的预测，不仅考虑当前时间'  步长的数据，还要考虑历史数据。（就是输出中包含很多类别的数据）\n    来源复杂\n\n输入然后得到的输出项（forward）\n\

#Model

In [24]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import accuracy_score
#感觉固定住了

class RNN_Model(nn.Module):
  def __init__(self):
    super(RNN_Model, self).__init__()
    self.rnn = nn.RNN(n_input, n_hidden, num_layers=2, batch_first=True, dropout=0.2)
    self.linear = nn.Linear(n_hidden, n_class) #output的线性层

  def forward(self, x):
    rnn_output, h_n = self.rnn(x)
    x_last = rnn_output[:,-1,:]
    x = self.linear(x_last)
    return x

model = RNN_Model().to(device)
criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(model.parameters(), lr= learning_rate)
#模型建立好了
#然后准备输入
input_batch, target_batch = make_batch(seq_data)
input_batch_torch = torch.from_numpy(np.array(input_batch)).float().to(device)  #把一个单词前三的字母的one-hot 张量形式放到一个list里面
target_batch_torch = torch.from_numpy(np.array(target_batch)).view(-1).to(device) #把每个单元都整合成了长单元
# to(device) 将输入转换为张量，并将其设置成为GPU 
# 参数-1 含义：自行计算行数或列数 


for epoch in range(total_epoch):
  model.train()  #将标志设置为训练模式；

  optimizer.zero_grad()
  outputs = model(input_batch_torch)    #output训练
  loss = criterion(outputs, target_batch_torch)
  loss.backward()
  optimizer.step()
  _, predicted = torch.max(outputs,1) #这个_代表的是什么

  acc = accuracy_score(predicted.cpu().numpy(),target_batch_torch.cpu().numpy()) #50个数据 epoch
  print('Epoch: %d, loss: %.5f, train_acc: %.2f' %(epoch+1, loss.item(),acc))

print('Finished Training')

##Prediction
model.eval()
outputs = model(input_batch_torch)
_, predicted = torch.max(outputs, 1)    # same as predicted = torch.argmax(pred_outputs, 1)

predict_words = []
for i in range(len(predicted.cpu().numpy())):
  ind = predicted.cpu().numpy()[i] #这步骤是让他变回数组而不是张量然后分别取出预测的每个值' index
  predict_words.append(seq_data[i][:-1]+char_arr[ind])  #他是去你除了最后一位的单词组合，然后将你预测的index放到对应的字母表中。从字母表中找到真实值。再将两个拼接
  print(predict_words)

print('\n=== Prediction Result ===')
print('Input:', [w[:3] + ' ' for w in seq_data])
print('Predicted:', predict_words)
print('Accuracy: %.2f' %acc)

Epoch: 1, loss: 3.25724, train_acc: 0.00
Epoch: 2, loss: 2.71709, train_acc: 0.50
Epoch: 3, loss: 2.08581, train_acc: 0.50
Epoch: 4, loss: 1.61668, train_acc: 0.50
Epoch: 5, loss: 1.41686, train_acc: 0.50
Epoch: 6, loss: 1.33283, train_acc: 0.50
Epoch: 7, loss: 1.33529, train_acc: 0.50
Epoch: 8, loss: 1.27455, train_acc: 0.60
Epoch: 9, loss: 1.20393, train_acc: 0.60
Epoch: 10, loss: 1.12294, train_acc: 0.50
Epoch: 11, loss: 1.05021, train_acc: 0.60
Epoch: 12, loss: 0.95618, train_acc: 0.70
Epoch: 13, loss: 0.88508, train_acc: 0.80
Epoch: 14, loss: 0.83438, train_acc: 0.80
Epoch: 15, loss: 0.74400, train_acc: 0.70
Epoch: 16, loss: 0.64337, train_acc: 0.80
Epoch: 17, loss: 0.64427, train_acc: 0.80
Epoch: 18, loss: 0.55463, train_acc: 0.80
Epoch: 19, loss: 0.45344, train_acc: 0.90
Epoch: 20, loss: 0.42318, train_acc: 0.90
Epoch: 21, loss: 0.35303, train_acc: 0.90
Epoch: 22, loss: 0.32012, train_acc: 0.90
Epoch: 23, loss: 0.26987, train_acc: 0.80
Epoch: 24, loss: 0.22863, train_acc: 0.90
E

#**Seq2Seq Model(N to M)**


In [25]:
'''
将一个序列转换为另一个序列。用RNN或者常用LSTM 或者GRU去避免梯度消失的问题。每个项目的上下文是上一步的输出。主要组件是一个编码器和一个解码器网络。
编码器将每个项目转换为包含该项目及其上下文的隐藏向量。
解码器使用先前的输出作为上下文来逆转该过程。将向量转换为输出项
'''

'\n将一个序列转换为另一个序列。用RNN或者常用LSTM 或者GRU去避免梯度消失的问题。每个项目的上下文是上一步的输出。主要组件是一个编码器和一个解码器网络。\n编码器将每个项目转换为包含该项目及其上下文的隐藏向量。\n解码器使用先前的输出作为上下文来逆转该过程。将向量转换为输出项\n'

**要开始一个新的项目了，该模型将纸牌符号（Ace，Jack，Queen，King） 转换为其相关编号**

In [26]:
'''
其实就两个模型一个是RNN，一个是Seq2Seq
1：处理数据。
2分批次
3模型
4评估

这个任务是"ace" 转变到"01" 将"jack"转变到"11"  【要求】老师
'''

'\n其实就两个模型一个是RNN，一个是Seq2Seq\n1：处理数据。\n2分批次\n3模型\n4评估\n\n这个任务是"ace" 转变到"01" 将"jack"转变到"11"  【要求】老师\n'

**Preprocess data**

In [27]:
'''
1调包
2 序列数据 sequence data
3 chars 字符  生成唯一的符号列表     tokens符号 
  每一个都拆开 + 去掉重复项
4 需要特殊的tokens 
    BEPU
    要额外添加
    B 序列开始
    E 序列结束
    P padding 填充（fill） 序列。 对于不同的输入大小
    U 序列中不知道的元素。 对于不同的输入大小

5 然后将字母和序号一一匹配上 ： n：i
6 记录整个长度
7 限定最大输入和输出的数量
'''

'\n1调包\n2 序列数据 sequence data\n3 chars 字符  生成唯一的符号列表     tokens符号 \n  每一个都拆开 + 去掉重复项\n4 需要特殊的tokens \n    BEPU\n    要额外添加\n    B 序列开始\n    E 序列结束\n    P padding 填充（fill） 序列。 对于不同的输入大小\n    U 序列中不知道的元素。 对于不同的输入大小\n\n5 然后将字母和序号一一匹配上 ： n：i\n6 记录整个长度\n7 限定最大输入和输出的数量\n'

In [64]:
import torch
import numpy as np
import pprint

seq_data = [['ace', '01'],['jack','11'],
            ['queen','12'],['king','13']]

chars =[] 
for seq in seq_data:
  chars += list(seq[0])
  chars += list(seq[1])

char_arr = list(set(chars)) #筛除重复的然后生成了列表 14

char_arr.append('B')
char_arr.append('E')
char_arr.append('P')
char_arr.append('U')    #18


num_dic = {n:i for i,n in enumerate(char_arr)} #对应字母：index 因为有0所以17 enumerate 就是让字母和数字一一对应

dic_len = len(num_dic)    #18

max_input_words_amount = 5
max_output_words_amount = 3

**产生batch--generate batch**

In [ ]:
'''
1.padding (fill 补充) 添满如果单词比最长的单词要短
2.产生一批数据，给训练和测试
    创建空白列表；编码输入；解码输入； 目标批次
    然后开始输入，将字典的中的word输入到input中
 然后找到固定的每个单词对应的index列表 P就是16

[5, 1, 0, 16, 16]
[11, 5, 1, 4, 16]
[8, 2, 0, 0, 3]
[4, 7, 3, 6, 16]

    左边的输入准备完了，然后要进入右边了，要先来个B  B + index
[14, 13, 9]
[14, 9, 9]
[14, 9, 10]
[14, 9, 12]

    B将index也转换成列表了
    但之后就取消了B，加了E 来让index1又转换成列表
[13, 9, 15]
[9, 9, 15]
[9, 10, 15]
[9, 12, 15]   
  
  所以右边是解码器；
    解码器单元的输出（实际结果），在序列数据的末尾添加E
    由于目标长度是固定的，但实际上我们  不需要在最后加E
    但不确定的目标长度，E就很重要了。
    因为在预测的过程中当涉及到E我们就要停

   然后开始将准备好的编码和解码输入变成 one-hot (逃不掉的one-hot)

3 这里输入的编码和解码（b） 都变成了one-hot存起来了
只有target （e） 存的是index，并没有存one-hot
'''

#这里是把词表输入。然后呢处理他们，得到 解码编码-onehot ；和目标（可能就是真实）index

In [104]:
def add_paddings(word, max_num=5):    #padding (fill 补充) 添满如果单词比最长的单词要短
  diff = max_num -len(word)
  return word + 'P'*diff    #用P补全  看起来要不补一个要不补两个P

def make_batch(seq_data):   #产生一批数据，给训练和测试
  encoder_input_batch = []
  decoder_input_batch = []
  target_batch = []

  for seq in seq_data:
    input_word = add_paddings(seq[0])     # 0 指前面的word 1 指后面的index  kingP
    en_input_data = [num_dic[n] for n in input_word]          #[5, 1, 0, 16, 16]
    
    de_input_data = [num_dic[n] for n in ('B' + seq[1])]  #B：14， 13：0； 9：1 它是把idnex也变成了列表

    target = [num_dic[n] for n in (seq[1] + 'E')]       #   13:0      E:15

    encoder_input_batch.append(np.eye(dic_len)) #就是五位。让他们在对应位置表1；  5；8；4；7；17 （还是需要减一，是从0开始；所以单独输出的还是King「 
                                                #让他们input的word变成one-hot
    decoder_input_batch.append(np.eye(dic_len)[de_input_data])    #np.eye(总单词长度)【实际你想要的变得单词index】
                                                #还是包含B的index
    target_batch.append(target)                 #包含E的,他是直接将输出的index存起来了，没做one-hot


  return encoder_input_batch, decoder_input_batch, target_batch

**Text**

In [103]:
target_batch.append(target)
target_batch

[[9, 12, 15], [9, 12, 15]]

In [100]:
#4.18
target

[9, 12, 15]

In [97]:
dic_len

18

In [98]:
np.eye(dic_len)[de_input_data]  #14, 9 ,12

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0.]])

In [95]:
en_input_data

[4, 7, 3, 6, 16]

In [94]:
#4.14
np.eye(dic_len)[en_input_data] #kingP 

array([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0.]])

In [93]:
#4.13
np.eye(dic_len)

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0

In [92]:
#4.12
encoder_input_batch = []
decoder_input_batch = []
target_batch = []

for seq in seq_data:
  input_word = add_paddings(seq[0])     # 0 指前面的word 1 指后面的index  kingP
  en_input_data = [num_dic[n] for n in input_word]          #[5, 1, 0, 16, 16]
  
  de_input_data = [num_dic[n] for n in ('B' + seq[1])]  #B：14， 13：0； 9：1 它是把idnex也变成了列表

  target = [num_dic[n] for n in (seq[1] + 'E')]
  print(target)

[13, 9, 15]
[9, 9, 15]
[9, 10, 15]
[9, 12, 15]


In [90]:
#4.11
seq

['king', '13']

In [88]:
#4.10
encoder_input_batch = []
decoder_input_batch = []
target_batch = []

for seq in seq_data:
  input_word = add_paddings(seq[0])     # 0 指前面的word 1 指后面的index  kingP
  en_input_data = [num_dic[n] for n in input_word]
  de_input_data = [num_dic[n] for n in ('B' + seq[1])]
  print(de_input_data)    

[14, 13, 9]
[14, 9, 9]
[14, 9, 10]
[14, 9, 12]


In [87]:
#4.9
encoder_input_batch = []
decoder_input_batch = []
target_batch = []

for seq in seq_data:
  input_word = add_paddings(seq[0])     # 0 指前面的word 1 指后面的index  kingP
  en_input_data = [num_dic[n] for n in input_word]
  print(en_input_data)    


[5, 1, 0, 16, 16]
[11, 5, 1, 4, 16]
[8, 2, 0, 0, 3]
[4, 7, 3, 6, 16]


In [79]:
#4.8 
n

'U'

In [77]:
#4.7
len(num_dic)

18

In [76]:
#4.6
num_dic[n]

17

In [75]:
#4.5
num_dic

{'0': 13,
 '1': 9,
 '2': 10,
 '3': 12,
 'B': 14,
 'E': 15,
 'P': 16,
 'U': 17,
 'a': 5,
 'c': 1,
 'e': 0,
 'g': 6,
 'i': 7,
 'j': 11,
 'k': 4,
 'n': 3,
 'q': 8,
 'u': 2}

In [74]:
input_word = add_paddings(seq[0])
input_word

'kingP'

In [70]:
seq[1]

'13'

In [69]:
#4.2
seq[0]

'king'

In [68]:
#4.1
seq_data

[['ace', '01'], ['jack', '11'], ['queen', '12'], ['king', '13']]

In [29]:
#3.1
chars += list(seq[0])
'''
['a', 'c', 'e']
['a', 'c', 'e', 'j', 'a', 'c', 'k']
['a', 'c', 'e', 'j', 'a', 'c', 'k', 'q', 'u', 'e', 'e', 'n']
['a', 'c', 'e', 'j', 'a', 'c', 'k', 'q', 'u', 'e', 'e', 'n', 'k', 'i', 'n', 'g']
'''
chars += list(seq[1])
'''
['a', 'c', 'e', '0', '1']
['a', 'c', 'e', '0', '1', 'j', 'a', 'c', 'k', '1', '1']
['a', 'c', 'e', '0', '1', 'j', 'a', 'c', 'k', '1', '1', 'q', 'u', 'e', 'e', 'n', '1', '2']
['a', 'c', 'e', '0', '1', 'j', 'a', 'c', 'k', '1', '1', 'q', 'u', 'e', 'e', 'n', '1', '2', 'k', 'i', 'n', 'g', '1', '3']

'''

"\n['a', 'c', 'e', '0', '1']\n['a', 'c', 'e', '0', '1', 'j', 'a', 'c', 'k', '1', '1']\n['a', 'c', 'e', '0', '1', 'j', 'a', 'c', 'k', '1', '1', 'q', 'u', 'e', 'e', 'n', '1', '2']\n['a', 'c', 'e', '0', '1', 'j', 'a', 'c', 'k', '1', '1', 'q', 'u', 'e', 'e', 'n', '1', '2', 'k', 'i', 'n', 'g', '1', '3']\n\n"

In [30]:
#2.17
seq_data

[['ace', '01'], ['jack', '11'], ['queen', '12'], ['king', '13']]

In [31]:
#2.16
char_arr

['e',
 'c',
 'u',
 'n',
 'k',
 'a',
 'g',
 'i',
 'q',
 '1',
 '2',
 'j',
 '3',
 '0',
 'B',
 'E',
 'P',
 'U']

In [32]:
#2.15
char_arr[ind]

'n'

In [33]:
#2.14
'''
['wor']
['wor', 'woo']
['wor', 'woo', 'dee']
['wor', 'woo', 'dee', 'div']
['wor', 'woo', 'dee', 'div', 'col']
['wor', 'woo', 'dee', 'div', 'col', 'coo']
['wor', 'woo', 'dee', 'div', 'col', 'coo', 'loa']
['wor', 'woo', 'dee', 'div', 'col', 'coo', 'loa', 'lov']
['wor', 'woo', 'dee', 'div', 'col', 'coo', 'loa', 'lov', 'kis']
['wor', 'woo', 'dee', 'div', 'col', 'coo', 'loa', 'lov', 'kis', 'kin']
'''

#seq_data[i][:-1]


"\n['wor']\n['wor', 'woo']\n['wor', 'woo', 'dee']\n['wor', 'woo', 'dee', 'div']\n['wor', 'woo', 'dee', 'div', 'col']\n['wor', 'woo', 'dee', 'div', 'col', 'coo']\n['wor', 'woo', 'dee', 'div', 'col', 'coo', 'loa']\n['wor', 'woo', 'dee', 'div', 'col', 'coo', 'loa', 'lov']\n['wor', 'woo', 'dee', 'div', 'col', 'coo', 'loa', 'lov', 'kis']\n['wor', 'woo', 'dee', 'div', 'col', 'coo', 'loa', 'lov', 'kis', 'kin']\n"

In [34]:
#2.13
ind

3

In [35]:
#2.12
predicted.cpu()[1]

tensor(3)

In [36]:
#2.11
predicted.cpu().numpy()

array([ 3,  3, 15,  4,  3, 11,  3,  4, 18,  3])

In [37]:
#2.10
predicted.cpu()

tensor([ 3,  3, 15,  4,  3, 11,  3,  4, 18,  3])

In [38]:
#2.9
predicted

tensor([ 3,  3, 15,  4,  3, 11,  3,  4, 18,  3], device='cuda:0')

In [39]:
#2.8
target_batch_torch.cpu().numpy()

array([ 3,  3, 15,  4,  3, 11,  3,  4, 18,  3])

In [40]:
#2.7
target_batch_torch

tensor([ 3,  3, 15,  4,  3, 11,  3,  4, 18,  3], device='cuda:0')

In [41]:
#2.6
print(target_batch_torch.cpu())#tensor([ 3,  3, 15,  4,  3, 11,  3,  4, 18,  3])


tensor([ 3,  3, 15,  4,  3, 11,  3,  4, 18,  3])


In [42]:
#2.5
  #print(_)    #tensor([0.2457, 0.1858, 0.2913, 0.1972, 0.2256, 0.2471, 0.2616, 0.1699, 0.2519,
              #0.2905], device='cuda:0', grad_fn=<MaxBackward0>)  [10]torch.Size([10]) 50+(不会是64吧）)
  #print(len(_))
  #print(predicted) #tensor([ 4,  4,  4,  4,  4,  4,  4,  4, 10,  4], device='cuda:0')
  #print(predicted.shape) #torch.Size([10])

In [43]:
#2.4
target_batch_torch

tensor([ 3,  3, 15,  4,  3, 11,  3,  4, 18,  3], device='cuda:0')

In [44]:
#2.3
target_batch

[[3], [3], [15], [4], [3], [11], [3], [4], [18], [3]]

In [45]:
#2.2
model.train()

RNN_Model(
  (rnn): RNN(26, 64, num_layers=2, batch_first=True, dropout=0.2)
  (linear): Linear(in_features=64, out_features=26, bias=True)
)

In [46]:
#2.1
input_batch_torch

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          1., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0

In [48]:
#1.9
'''
target_batch = []
for seq in seq_data:
  input_data = [num_dic[n] for n in seq[:-1]] ##不取最后一个，把前面的字母对应的index取出来
  target = num_dic[seq[-1]]

  input_batch.append(np.eye(dic_len)[input_data])

  target_batch.append([target])
  print(target_batch)
'''

'\ntarget_batch = []\nfor seq in seq_data:\n  input_data = [num_dic[n] for n in seq[:-1]] ##不取最后一个，把前面的字母对应的index取出来\n  target = num_dic[seq[-1]]\n\n  input_batch.append(np.eye(dic_len)[input_data])\n\n  target_batch.append([target])\n  print(target_batch)\n'

In [50]:
#1.8
'''
import pprint
input_batch = []
for seq in seq_data:
  input_data = [num_dic[n] for n in seq[:-1]] ##不取最后一个，把前面的字母对应的index取出来

  input_batch.append(np.eye(dic_len)[input_data]) #2-3【0】【0】 还是个list；还没有形状0 0
  

'''

'\nimport pprint\ninput_batch = []\nfor seq in seq_data:\n  input_data = [num_dic[n] for n in seq[:-1]] ##不取最后一个，把前面的字母对应的index取出来\n\n  input_batch.append(np.eye(dic_len)[input_data]) #2-3【0】【0】 还是个list；还没有形状0 0\n  \n\n'

In [52]:
#np.eye(dic_len)[input_data] #制作one-hot np.eye 批量制作。然后第一个是0.所以index+1的位置是1.代表了这个元素

In [63]:
#input_data #kid

In [ ]:
np.eye(dic_len)#弄了个对角线1其他都是0


In [62]:
#1.7
'''
for seq in seq_data:
  target = num_dic[seq[-1]]
  print(target)     #3 :d  
  '''

'\nfor seq in seq_data:\n  target = num_dic[seq[-1]]\n  print(target)     #3 :d  \n  '

In [61]:
#1.6
'''
for seq in seq_data:
    input_data = [num_dic[n] for n in seq[:-1]] #不取最后一个
    print(input_data)
    #w:22 o:14 r:17
    '''

'\nfor seq in seq_data:\n    input_data = [num_dic[n] for n in seq[:-1]] #不取最后一个\n    print(input_data)\n    #w:22 o:14 r:17\n    '

In [58]:
#1.5
seq_data
len(seq_data)

4

In [57]:
#1.4
dic_len

18

In [56]:
#1.3
{n:i for i, n in enumerate(char_arr)} #   n: i == 'a': 0,单词： index 目前看起来只有字典可以这样

{'0': 13,
 '1': 9,
 '2': 10,
 '3': 12,
 'B': 14,
 'E': 15,
 'P': 16,
 'U': 17,
 'a': 5,
 'c': 1,
 'e': 0,
 'g': 6,
 'i': 7,
 'j': 11,
 'k': 4,
 'n': 3,
 'q': 8,
 'u': 2}

In [55]:
#1.2
for i,n in enumerate(char_arr):
  #print(i,)    #index
  #print(n)    #word
  print(i,n)  #这样是一一对应（横向）

0 e
1 c
2 u
3 n
4 k
5 a
6 g
7 i
8 q
9 1
10 2
11 j
12 3
13 0
14 B
15 E
16 P
17 U


In [54]:
#1.1
list(enumerate(char_arr))#这没什么变化啊


[(0, 'e'),
 (1, 'c'),
 (2, 'u'),
 (3, 'n'),
 (4, 'k'),
 (5, 'a'),
 (6, 'g'),
 (7, 'i'),
 (8, 'q'),
 (9, '1'),
 (10, '2'),
 (11, 'j'),
 (12, '3'),
 (13, '0'),
 (14, 'B'),
 (15, 'E'),
 (16, 'P'),
 (17, 'U')]